# L1 and L2 Regularization

## Implementation 

In [1]:
import nnfs
import numpy as np
from nnfs.datasets import spiral_data

In [2]:
nnfs.init()

In [3]:
import os
import sys

module = os.path.abspath(os.path.join(".."))
if module not in sys.path:
    sys.path.append(module)

In [4]:
from Updated_Classes import *

In [5]:
X, y = spiral_data(samples=1000, classes=3)

In [6]:
dense1 = Layer_Dense(2, 512, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(512, 3)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

In [7]:
from Chpt_10.Optimizers import Optimizer_Adam

In [8]:
optimizer = Optimizer_Adam(learning_rate=0.02, decay=5e-7)

In [9]:
for epoch in range(10001):

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    data_loss = loss_activation.forward(dense2.output, y)

    regularization_loss = loss_activation.loss.regularization_loss(
        dense1
    ) + loss_activation.loss.regularization_loss(dense2)

    loss = data_loss + regularization_loss

    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(
            f"epoch: {epoch}, "
            + f"acc: {accuracy:.3f}, "
            + f"loss: {loss:.3f} ("
            + f"data_loss: {data_loss:.3f}, "
            + f"reg_loss: {regularization_loss:.3f}), "
            + f"lr: {optimizer.current_learning_rate}"
        )

    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.381, loss: 1.099 (data_loss: 1.099, reg_loss: 0.000), lr: 0.02
epoch: 100, acc: 0.759, loss: 0.734 (data_loss: 0.680, reg_loss: 0.054), lr: 0.019999010049002574
epoch: 200, acc: 0.825, loss: 0.562 (data_loss: 0.474, reg_loss: 0.088), lr: 0.019998010197985302
epoch: 300, acc: 0.856, loss: 0.488 (data_loss: 0.389, reg_loss: 0.098), lr: 0.019997010446938183
epoch: 400, acc: 0.862, loss: 0.452 (data_loss: 0.353, reg_loss: 0.099), lr: 0.01999601079584623
epoch: 500, acc: 0.891, loss: 0.422 (data_loss: 0.325, reg_loss: 0.097), lr: 0.01999501124469445
epoch: 600, acc: 0.895, loss: 0.396 (data_loss: 0.303, reg_loss: 0.093), lr: 0.01999401179346786
epoch: 700, acc: 0.903, loss: 0.380 (data_loss: 0.290, reg_loss: 0.090), lr: 0.01999301244215147
epoch: 800, acc: 0.898, loss: 0.370 (data_loss: 0.284, reg_loss: 0.086), lr: 0.0199920131907303
epoch: 900, acc: 0.808, loss: 0.605 (data_loss: 0.516, reg_loss: 0.088), lr: 0.019991014039189386
epoch: 1000, acc: 0.908, loss: 0.357 (data_l

## Validation

In [10]:
X_test, y_test = spiral_data(samples=100, classes=3)

In [12]:
dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.output, y_test)

In [13]:
predictions = np.argmax(loss_activation.output, axis=1)
if len(y.shape) == 2:
    y_test = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test)

In [14]:
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}')

validation, acc: 0.920, loss: 0.256
